# Part I. ETL Pipeline for Pre-Processing the Files

## Preprocessing Files From CSV

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/johnrick/Desktop/udacity/cassandra_project


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# create pandas 
df = pd.read_csv('event_datafile_new.csv')

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


# Task 1: Find the artist, song title and songs length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

## Create song_info_for_itemInSession Table

In [9]:
# Create table that includes: artist, song, song length, sessionID, itemInSession
session.execute('''CREATE TABLE IF NOT EXISTS song_info_for_itemInSession 
    (
        sessionId INT, 
        itemNoInSession INT, 
        artist TEXT, 
        song_title TEXT, 
        song_length float, 
        PRIMARY KEY(sessionId, itemNoInSession)
    );''')

## Insert Into Table

In [10]:
# save csv to a variable
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = """INSERT INTO song_info_for_itemInSession 
            (
                sessionId, 
                itemNoInSession, 
                artist, 
                song_title, 
                song_length
            ) 
            VALUES (%s, %s, %s, %s, %s)"""
        # Insert statement using column location
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

## Run Query

In [11]:
## Get artist, song, and song length for sessionId and itemInSession
rows = session.execute("SELECT artist, song_title, song_length FROM song_info_for_itemInSession WHERE sessionId = 338 AND itemNoInSession =4;")

## Results

In [12]:
# view answer
pd.DataFrame(rows)

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


# Task 2: Find the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182  

## Create song_playlist_session table

In [13]:
# Create table
session.execute("""CREATE TABLE IF NOT EXISTS song_playlist_session 
    (
        userId INT, 
        sessionId INT, 
        itemInSession INT, 
        artist TEXT, 
        song TEXT, 
        firstName TEXT, 
        lastName TEXT, 
        PRIMARY KEY (userId, sessionId, itemInSession)
    );""")

## Insert Information Into Table

In [14]:
# Save csv into variable
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO song_playlist_session 
            (
                userId, 
                sessionId, 
                itemInSession, 
                artist, 
                song, 
                firstName, 
                lastName
            ) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

## Run Query

In [15]:
## query2: Artist, song (sorted by itemInSession) and user (for userid = 10, sessionid = 182
rows = session.execute("SELECT artist, song, firstName, lastName FROM song_playlist_session WHERE userId = 10 AND sessionId = 182;")

## Results

In [16]:
pd.DataFrame(rows)

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


# Task 3: Find every user in the music app history who listened to the song 'All Hands Against His Own'

## Create user_history_for_song

In [17]:
# user_history: userId, first, last, song
session.execute('''CREATE TABLE IF NOT EXISTS user_history_for_song
    (
        song TEXT, 
        userId INT, 
        firstName TEXT, 
        lastName TEXT, 
        PRIMARY KEY(song, userId)
    );''')

## Insert Information Into Table

In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO user_history_for_song 
            (
                song, 
                userId, 
                firstName, 
                lastName
            ) 
        VALUES (%s, %s, %s, %s)"""
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

## Run Query

In [19]:
# every user name (first and last) in history who listened to the song 'All Hands Against His Own'                    
rows = session.execute("SELECT firstName, lastName FROM user_history_for_song WHERE song = 'All Hands Against His Own'")

## Results

In [20]:
pd.DataFrame(rows)

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


# Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions
drop_table1 = "DROP TABLE IF EXISTS song_info_for_itemInSession"
drop_table2 = "DROP TABLE IF EXISTS song_playlist_session"
drop_table3 = "DROP TABLE IF EXISTS user_history_for_song"
drop_tables = [drop_table1, drop_table2, drop_table3]

for i in drop_tables:
    try: 
        session.execute(i)
    except Exception as e:
        print(e)

# Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()